In [1]:
from __future__ import absolute_import

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchsummary import summary

from my_efficientnet_pytorch_3d import EfficientNet3D

import scipy.ndimage as nd

In [2]:
import os
import platform
# from os import listdir
import pandas as pd
import numpy as np
# import glob
# import tqdm
from typing import Dict
# import matplotlib.pyplot as plt
# %matplotlib inline

In [3]:
IMAGE_PATH = "../input/osic-pulmonary-fibrosis-progression/" if 'linux' in platform.platform().lower() else 'data/'

train_df = pd.read_csv(f'{IMAGE_PATH}/train.csv')
test_df = pd.read_csv(f'{IMAGE_PATH}/test.csv')

# print(Fore.YELLOW + 'Training data shape: ',Style.RESET_ALL,train_df.shape)
# train_df.head(5)

In [4]:
# Creating unique patient lists and their properties. 
train_dir = f'{IMAGE_PATH}/train/'
test_dir = f'{IMAGE_PATH}/test/'

patient_ids = os.listdir(train_dir)
patient_ids = sorted(patient_ids)

#Creating new rows
no_of_instances = []
age = []
sex = []
smoking_status = []

for patient_id in patient_ids:
    patient_info = train_df[train_df['Patient'] == patient_id].reset_index()
    no_of_instances.append(len(os.listdir(train_dir + patient_id)))
    age.append(patient_info['Age'][0])
    sex.append(patient_info['Sex'][0])
    smoking_status.append(patient_info['SmokingStatus'][0])

#Creating the dataframe for the patient info    
patient_df = pd.DataFrame(list(zip(patient_ids, no_of_instances, age, sex, smoking_status)), 
                                 columns =['Patient', 'no_of_instances', 'Age', 'Sex', 'SmokingStatus'])
# print(patient_df.info())
# patient_df.head()

In [5]:
answer = []
for i, patient in test_df.iterrows():
    for week in range(-12, 133 + 1):
        tmp_id = patient.Patient + '_' + str(week)
        FVC = patient.FVC + (-110 if week > 0 else 130)
        Confidence = 70
        answer.append([tmp_id, FVC, Confidence])
#         print(tmp_id, FVC)

In [6]:
result = pd.DataFrame(answer, columns=['Patient_Week', 'FVC', 'Confidence'])

In [7]:
# result

In [8]:
result.to_csv("submission.csv", index=False)  # .to_csv('sample_submission.csv', sep=',')

In [9]:
# %config Completer.use_jedi = False

In [10]:
import os
import sys

import cv2

import scipy

import skimage
import skimage.measure
import skimage.filters
import skimage.segmentation

import imageio

import tabulate

import numpy as np

import tqdm.notebook as tqdm

from collections import defaultdict

from pydicom import dcmread
from pydicom import multival

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

from IPython.display import clear_output, display

In [11]:
# image = dcmread(f'{IMAGE_PATH}/train/ID00183637202241995351650/52.dcm')
# image
# plt.imshow(image.pixel_array, cmap="gray")


# def subplots_3d(nrows=1, ncols=1, figsize=None):
#     fig, axes = plt.subplots(nrows, ncols, figsize=figsize)
#     axes = np.array(axes).reshape([nrows, ncols])
    
#     for idx in range(len(axes)):
#         for jdx in range(len(axes[idx])):
#             axes[idx][jdx].remove()
#             axes[idx][jdx] = fig.add_subplot(nrows, ncols, 1 + idx * ncols + jdx,projection='3d')
            
#     if nrows == 1:
#         return fig, axes[0].tolist()
#     return fig, axes.tolist()

# def plot_3d(ax, image, stride, threshold=700, color="navy"):
#     if isinstance(stride, (int, float)):
#         stride = (stride, stride, stride)
    
#     image = image[::stride[0], ::stride[1], ::stride[2]]
        
#     # Position the scan upright, 
#     # so the head of the patient would be at the top facing the camera
#     p = image.transpose(2,1,0)
    
#     verts, faces,_,_ = skimage.measure.marching_cubes(p, threshold)
    
#     # Fancy indexing: `verts[faces]` to generate a collection of triangles
#     mesh = Poly3DCollection(verts[faces], alpha=0.2)
#     mesh.set_facecolor(color)
#     ax.add_collection3d(mesh)

#     ax.set_xlim(0, p.shape[0])
#     ax.set_ylim(0, p.shape[1])
#     ax.set_zlim(0, p.shape[2])
    
# def sample_slices(images, n_slices, masks=None, alpha=0.2, cmap_name=None):
#     nrows, ncols = n_slices // 4 + ((n_slices % 4) > 0), 4
#     fig, axes = plt.subplots(nrows, ncols, figsize=(ncols * 5, nrows * 4))
#     axes = np.array(axes).reshape(-1)

#     idxs = np.sort(np.random.choice(np.arange(len(images)), nrows * ncols, replace=False))
    
#     for ax, idx in zip(axes, idxs):
#         ax.get_xaxis().set_visible(False)
#         ax.get_yaxis().set_visible(False)
#         ax.imshow(images[idx], cmap=plt.get_cmap(cmap_name))
#         if masks is not None:
#             ax.imshow(masks[idx], alpha=alpha)
# #             ax.imshow(masks[idx], alpha=alpha, cmap=plt.get_cmap(cmap_name))
#         ax.grid(False)
#         ax.set_title(f'Slice {idx}')
#     plt.show()

In [12]:
# test_patients = sorted(os.listdir('./data/test/'))
# train_patients = sorted(os.listdir('./data/train/'))

In [13]:
# test_ct = {
#     patient: [
#         dcmread(os.path.join('./data/test', patient, ct_file)) 
#         for ct_file in sorted(os.listdir(os.path.join('./data/test/', patient)))
#     ] for patient in test_patients
# }
# train_ct = {
#     patient: [
#         dcmread(os.path.join('./data/train', patient, ct_file)) 
#         for ct_file in os.listdir(os.path.join('./data/train/', patient))
#     ] for patient in train_patients
# }

In [14]:
def getkey(dcm, key):
    try:
        base_value = dcm[key]
    except:
        print(dcm.PatientID, dcm.InstanceNumber, key)
        return'KeyError'
    try:
        return base_value.value
    except:
        return base_value

def transform_to_hu(ct):
    slope = ct.RescaleSlope
    intercept = ct.RescaleIntercept
    image = ct.pixel_array.astype(np.int16)
    
    # some images has ousige pixel-values corresponging water
    # fix those images
    zero_cols = np.argwhere(np.sum(ct.pixel_array, axis=0) == 0).reshape(-1)
    zero_rows = np.argwhere(np.sum(ct.pixel_array, axis=1) == 0).reshape(-1)
    image[zero_rows, :] = -1000 - np.int16(intercept)
    image[:, zero_cols] = -1000 - np.int16(intercept)
    
    # convert to HU
    if slope != 1:
        image = (slope * image.astype(np.float64)).astype(np.int16)
    image += np.int16(intercept)
    
    # convert ouside pixel-values to air:
    # I'm using <= -1000 to be sure that other defaults are captured as well
    image[image <= -1000] = -1000
    
    return ct, image

def crop(image, center, size):
    return image.copy()[
        center[0] - size[0] // 2:center[0] + size[0] // 2
    ][:, center[1] - size[1] // 2:center[1] + size[1] // 2]

transformations_in_train = defaultdict(lambda: lambda x: x, {
    'ID00014637202177757139317': lambda x: crop(x, (x.shape[0] // 2, x.shape[1] // 2), (512, 512)),
    'ID00067637202189903532242': lambda x: crop(x, (x.shape[0] // 2, x.shape[1] // 2), (512, 512)),
    'ID00086637202203494931510': lambda x: crop(x, (x.shape[0] // 2, x.shape[1] // 2), (512, 512)),
    'ID00094637202205333947361': lambda x: crop(x, (x.shape[0] // 2, x.shape[1] // 2), (512, 512)),
    'ID00122637202216437668965': lambda x: crop(x, (x.shape[0] // 2, x.shape[1] // 2), (512, 512)),
    'ID00240637202264138860065': lambda x: crop(x, (x.shape[0] // 2, x.shape[1] // 2), (512, 512)),
    'ID00419637202311204720264': lambda x: crop(x, (x.shape[0] // 2, x.shape[1] // 2), (512, 512))
})

# # images that needs transformation in train set:
# '''
#     ID00115637202211874187958 1302 1302
#     ID00288637202279148973731 632 632
#     ID00358637202295388077032 632 632 
#     ID00009637202177434476278 768 768
#     ID00015637202177877247924 768 768
#     ID00025637202179541264076 768 768
#     ID00026637202179561894768 768 768
#     ID00027637202179689871102 768 768
#     ID00038637202182690843176 768 768
#     ID00042637202184406822975 768 768
#     ID00078637202199415319443 768 768
#     ID00082637202201836229724 768 768
#     ID00089637202204675567570 768 768
#     ID00105637202208831864134 768 768
#     ID00108637202209619669361 768 768
#     ID00110637202210673668310 768 768
#     ID00128637202219474716089 768 768
#     ID00129637202219868188000 768 768
#     ID00132637202222178761324 768 768
#     ID00169637202238024117706 768 768
#     ID00173637202238329754031 768 768
#     ID00183637202241995351650 768 768
#     ID00214637202257820847190 768 768
#     ID00216637202257988213445 768 768
#     ID00242637202264759739921 768 768
#     ID00248637202266698862378 768 768
#     ID00285637202278913507108 768 768
#     ID00290637202279304677843 768 768
#     ID00291637202279398396106 768 768
#     ID00309637202282195513787 768 768
#     ID00343637202287577133798 768 768
#     ID00344637202287684217717 768 768
#     ID00351637202289476567312 768 768
#     ID00367637202296290303449 768 768
#     ID00388637202301028491611 768 768
#     ID00414637202310318891556 768 768
#     ID00421637202311550012437 768 768
# '''

transformations_in_test = defaultdict(lambda: lambda x: x, {
    'ID00419637202311204720264': lambda x: crop(x, (x.shape[0] // 2, x.shape[1] // 2), (512, 512))
})
    
# # images that needs transformation in test set:
# '''
#     ID00421637202311550012437 768 768
# '''

transformations = {
    'test': transformations_in_test,
    'train': transformations_in_train
}

def int16touint8(image):
    result = image.copy().astype(np.int16)
    result = result - result.min() + 10
    result = result / (result.max() + 10)
    return (result * 255.0).astype(np.uint8)

def segmentate_patient(mode, patient_n):
    base_path = os.path.join('./data', mode)
    patient = sorted(os.listdir(base_path))[patient_n]
    patient_path = os.path.join(base_path, patient)
    
    all_images, all_lungs, all_residuals, all_masks = [], [], [], []
    meta_data = {
        'InstanceNumber': [], 'SliceLocation': [], 'SliceThickness': [], 'PixelSpacing': [], 'PatientPosition': []
    }
    for idx, ct_name in enumerate(sorted(os.listdir(patient_path), key=lambda x: int(x.split('.')[0]))):
        ct_path = os.path.join(patient_path, ct_name)
        ct, ct_image = transform_to_hu(dcmread(ct_path))
        ct_image = transformations[mode][patient](ct_image)
        all_images.append(ct_image)
        
        lungs, residual, mask = segment_lungs(ct_image, patient_n, display=False)
        
        all_lungs.append(lungs)
        all_residuals.append(residual)        
        all_masks.append(mask)
        
        for key in meta_data.keys():
            meta_data[key].append(getkey(ct, key))
            
    return all_images, all_lungs, all_residuals, all_masks, meta_data

def merge(all_lungs, all_residuals, meta_data):
    def _issorted(_array):
        return np.all(_array[:-1] <= _array[1:]) or np.all(_array[:-1] >= _array[1:])
    
    SliceLocations = np.array(meta_data['SliceLocation'])
    InstanceNumbers = np.array(meta_data['InstanceNumber'])
    
#     assert _issorted(SliceLocations)
#     assert _issorted(InstanceNumbers)
    
    return np.stack(all_lungs), np.stack(all_residuals)

def _get_max_quantile(_array, _thresh=0.9):
    sorted_array = np.sort(_array)[::-1]
    thresh_idx =  np.argwhere((np.cumsum(sorted_array) / np.sum(_array) > _thresh)).reshape(-1)[0]
    return sorted_array[thresh_idx]

def segment_lungs(image, patient_n, display=False):
    def _contour_border_distance(_contour, shape):
        _distance = 2 * shape[0]
        _vdistance = 2 * shape[0]
        _hdistance = 2 * shape[0]
        for (point, ) in _contour:
            # image[point[1], point[0]]
            _distance = min(_distance, point[0], point[1], shape[1] - point[0], shape[0] - point[1])
            _vdistance = min(_vdistance, point[1], shape[0] - point[1])
            _hdistance = min(_hdistance, point[0], shape[1] - point[0])
        return _distance, _vdistance, _hdistance
    
    def _filter_contours(_mask):
        # Find all segments on the mask
        contours, hierarchy = cv2.findContours(_mask.astype(np.uint8), 1, 2)
        if len(contours) == 0:
            return _mask
        
        # Evaluate some statistics for each segment like area, perimeter, shape of bounding box, distance to borders
        countours_metrics = []
        for contour in contours:
            ((_, _), (horizontal_range, vertical_range), _) = cv2.minAreaRect(contour)

            countours_metrics.append(
                [
                    horizontal_range, vertical_range, 
                    cv2.arcLength(contour, True), cv2.contourArea(contour), 
                    _contour_border_distance(contour, image.shape)
                ]
            )
        
        # If a perimeter to small -- drop segment
        perimeter_thresh_1 = 2 * (2 * (_mask.shape[0] / 512.0) + 2 * (_mask.shape[1] / 512.0))
        # If a perimeter less than 10-th quantile -- drop segment
        perimeter_thresh_2 = np.quantile([_[2] for _ in countours_metrics], 0.1) if len(countours_metrics) > 7 else 0.0
        perimeter_thresh = max(perimeter_thresh_1, perimeter_thresh_2)

        # If an area to small -- drop segment
        area_thresh_1 = 6 * 6 * (_mask.shape[0] / 512.0)  * (_mask.shape[1] / 512.0)
        # If anarea less than 10-th quantile -- drop segment
        area_thresh_2 = np.quantile([_[3] for _ in countours_metrics], 0.1) if len(countours_metrics) > 7 else 0.0
        # Save the lagrest segments -- segments that overlap more than 95% of area of all segments
        area_thresh_3 = _get_max_quantile([_[3] for _ in countours_metrics], 0.95)
        area_thresh = max(area_thresh_1, area_thresh_2, area_thresh_3)
        
        # If a segment too close to the top or bottom border -- drop segment
        vdistance_to_border_thresh = 30.0 * (_mask.shape[0] / 512.0)
        # If a segment too close to the left or right border -- drop segment
        hdistance_to_border_thresh = 3.0 * (_mask.shape[1] / 512.0)
        
        for contour, (horizontal_range, vertical_range, perimeter, area, distances) in zip(contours, countours_metrics):
            distance, vdistance, hdistance = distances
            if_small_area = area < area_thresh
            if_small_high = vertical_range < 2.0
            if_small_width = horizontal_range < 2.0
            if_small_perimeter = perimeter < perimeter_thresh
            if_too_close_to_vborder = vdistance < vdistance_to_border_thresh
            if_too_close_to_hborder = hdistance < hdistance_to_border_thresh
            # If a segment too wide in compare to its height -- drop segment 
            if_too_long_1 = max(horizontal_range / vertical_range, vertical_range / horizontal_range) > 7.5
            # If a segment too wide and too flat -- drop segment 
            if_too_long_2 = horizontal_range > 0.5 * _mask.shape[0] and vertical_range < 0.15 * _mask.shape[1]
            # If a segment really too wide -- drope segment
            if_too_long_3 = horizontal_range > 0.87 * _mask.shape[0] or vertical_range > 0.87 * _mask.shape[1]
            
            if (
                if_small_area or 
                if_small_high or 
                if_small_width or 
                if_small_perimeter or
                if_too_close_to_vborder or
                if_too_close_to_hborder or
                if_too_long_1 or
                if_too_long_2 or 
                if_too_long_3
            ):        
                _mask = cv2.drawContours(
                    _mask.astype(np.uint8), [contour],
                    contourIdx=-1, color=(0), thickness=-1).astype(np.bool)
                if display:
                    print('Remove: ', (horizontal_range, vertical_range, perimeter, area, distances))
                    table = tabulate.tabulate([[
                        if_small_area,
                        if_small_high,
                        if_small_width,
                        if_small_perimeter,
                        if_too_close_to_vborder,
                        if_too_close_to_hborder,
                        if_too_long_1,
                        if_too_long_2,
                        if_too_long_3
                    ]], headers=[
                        'smll_area',
                        'smll_high',
                        'smll_width',
                        'smll_perimeter',
                        'close_to_vbrd',
                        'close_to_hbrd',
                        'long_1',
                        'long_2',
                        'long_3'
                    ])
                    print(table)
            else:
                if display:
                    print('Do not remove: ', (horizontal_range, vertical_range, perimeter, area, distances))
        
        return _mask

    thresh = -700
    thresh_mask = image <= thresh
    
    if patient_n >= 26:
        # Remove small holes and disconnections (Fixes patient №26. Need to check first 26 patients)
        # Bad for some patients with № < 26
        kernel = skimage.morphology.disk(3)
        thresh_mask = skimage.morphology.binary_closing(thresh_mask, selem=kernel)
    
    thresh_mask = skimage.segmentation.clear_border(thresh_mask)

    # Smooth image with
    lungs_mask = skimage.filters.median(thresh_mask)
    
    # Remove small holes and disconnections
    kernel = skimage.morphology.disk(7)
    lungs_mask = skimage.morphology.binary_closing(lungs_mask, selem=kernel)
    
    # Expand verticaly
    kernel = np.ones([3, 1], dtype=np.bool)
    lungs_mask = skimage.morphology.dilation(lungs_mask, selem=kernel)
    # Expand horizontaly
    kernel = np.ones([1, 7], dtype=np.bool)
    lungs_mask = skimage.morphology.dilation(lungs_mask, selem=kernel)
    lungs_mask = skimage.morphology.dilation(lungs_mask, selem=kernel)
    lungs_mask = skimage.morphology.dilation(lungs_mask, selem=kernel)
    
    # Remove some garbage
    lungs_mask = _filter_contours(lungs_mask)
    
    # Squeeze image horizontaly back
    kernel = np.ones([1, 7], dtype=np.bool)
    lungs_mask = skimage.morphology.erosion(lungs_mask, selem=kernel)
    lungs_mask = skimage.morphology.erosion(lungs_mask, selem=kernel)
    lungs_mask = skimage.morphology.erosion(lungs_mask, selem=kernel)
    # Squeeze image verticaly back
    kernel = np.ones([3, 1], dtype=np.bool)
    lungs_mask = skimage.morphology.erosion(lungs_mask, selem=kernel)
    
    # Fixes patient №55 (works well for all patients)
    kernel = np.ones([3, 1], dtype=np.bool)
    lungs_mask = skimage.morphology.erosion(lungs_mask, selem=kernel)
    # Remove small holes
    lungs_mask = scipy.ndimage.binary_fill_holes(lungs_mask)
    # Fixes patient №55 (works well for all patients)
    kernel = np.ones([3, 1], dtype=np.bool)
    lungs_mask = skimage.morphology.dilation(lungs_mask, selem=kernel)
    
    lungs = lungs_mask * image
    residual = (1.0 - lungs_mask) * image
    
    lungs[lungs == 0] = np.min(image)
    residual[residual == 0] = np.min(image)
    
    if display:
        fig, ax = plt.subplots(1, 5, figsize=(20, 15))

        ax[0].set_title('HU Image')
        ax[0].imshow(image, cmap='gray')
        ax[0].axis('off')

        ax[1].set_title('Thresholded Image')
        ax[1].imshow(thresh_mask, cmap='gray')
        ax[1].axis('off')

        ax[2].set_title('Lungs Mask')
        ax[2].imshow(lungs_mask)
        ax[2].axis('off')

        ax[3].set_title('Lungs Image')
        ax[3].imshow(lungs)
        ax[3].axis('off')
        
        ax[4].set_title('Residual Image')
        ax[4].imshow(residual)
        ax[4].axis('off')
    
    return lungs, residual, lungs_mask



params = {
    'base_threshold': -700,
    'min_vperimeter': 2, 'min_hperimeter': 2,
    'min_varea': 6, 'min_harea': 6,
    'min_quantile_contours': 7,
    'perimeter_quantile': 0.1,
    'area_quantile': 0.1, 'area_max_quantile': 0.95,
    'min_vdistance': 30.0, 'min_hdistance': 3.0,
    'min_hv_ratio': 7.5
}

'''
    ID00026637202179561894768 №11 -- strange images
    ID00076637202199015035026 №30 -- first 9 images need to be fixed
    ID00078637202199415319443 №32 -- two breathes
    ID00123637202217151272140 №51 -- very bag segmentation
    ID00126637202218610655908 №54 -- first 1 images need to be fixed
    ID00128637202219474716089 №56 -- strange images
    ID00129637202219868188000 №57 -- very bag segmentation
'''

'\n    ID00026637202179561894768 №11 -- strange images\n    ID00076637202199015035026 №30 -- first 9 images need to be fixed\n    ID00078637202199415319443 №32 -- two breathes\n    ID00123637202217151272140 №51 -- very bag segmentation\n    ID00126637202218610655908 №54 -- first 1 images need to be fixed\n    ID00128637202219474716089 №56 -- strange images\n    ID00129637202219868188000 №57 -- very bag segmentation\n'

In [15]:
# mode, patient_n = 'train', 32
# all_images, all_lungs, all_residuals, all_masks, meta_data = segmentate_patient(mode, patient_n)
# lungs_3d, residuals_3d = merge(all_lungs, all_residuals, meta_data)

# print(f'{mode} patient {patient}. number of slices: {len(all_images)}')

# _, __, ___ = segment_lungs(all_images[0], patient_n, display=True)
# _, __, ___ = segment_lungs(all_images[len(all_images) // 2], patient_n, display=True)
# _, __, ___ = segment_lungs(all_images[-1], patient_n, display=True)
# # _, __, ___ = segment_lungs(all_images[24], patient_n, display=True)

In [16]:
# n_samples = len(all_images) // 2 if len(all_images) > 200 else len(all_images) - 3
# sample_slices(all_images, n_samples, masks=all_masks, alpha=.2, cmap_name='gray')
# # sample_slices(all_lungs, 28, masks=None, alpha=0.2, cmap_name=None)
# # sample_slices(all_images[:28], 28, masks=None, alpha=0.2, cmap_name=None)
# # sample_slices(all_masks[:28], 28, masks=None, alpha=0.2, cmap_name=None)

In [17]:
# fig, axes = subplots_3d(1, 2, figsize=(15, 7))

# plot_3d(axes[0], lungs_3d, stride=1, threshold=-900)
# plot_3d(axes[1], residuals_3d, stride=1, threshold=700)

# plt.show()

In [18]:
def resample(image, SliceThickness, PixelSpacing, new_spacing=(1.0, 1.0, 1.0)):
    # Determine current pixel spacing
    spacing = np.array([SliceThickness] + list(PixelSpacing), dtype=np.float32)

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor, mode='nearest')
    
    return image, new_spacing

In [19]:
# lungs_3d_resampled, new_spacing = resample(lungs_3d, meta_data['SliceThickness'][-1], meta_data['PixelSpacing'][-1])

# lungs_3d.shape, lungs_3d_resampled.shape

# sample_slices(lungs_3d_resampled, 10)

# gif_writer = imageio.get_writer('./test.gif', mode='I', duration=0.1) 
# for _ in tqdm.tqdm(range(1, 60)):
#     ct, ct_image = transform_to_hu(dcmread(os.path.join('./data/train', 'ID00421637202311550012437', f'{_}.dcm')))
#     lungs, lungs_mask = segment_lungs(ct_image)
#     gif_writer.append_data(np.hstack([int16touint8(ct_image), int16touint8(lungs), int16touint8(lungs_mask)]))

# plt.imshow(lung_segment(ct_image)[0]); plt.show()
# plt.imshow(lung_segment(ct_image)[1]); plt.show()

In [20]:
# mode = 'train'
# base_path = os.path.join('./data', mode)

# counter = defaultdict(int)
# for patient in tqdm.tqdm(os.listdir(base_path)):
#     patient_counter = defaultdict(int)
#     patient_path = os.path.join(base_path, patient)
#     for idx, ct_name in enumerate(sorted(os.listdir(patient_path), key=lambda x: int(x.split('.')[0]))):
#         if idx > -1:
#             ct_path = os.path.join(patient_path, ct_name)
#             ct, ct_image = transform_to_hu(dcmread(ct_path))
#             ct_image = transformations[mode][patient](ct_image)
#             patient_counter[(getkey(ct, 'PixelSpacing')[0])] += 1
            
#             del ct
    
#     for key in patient_counter.keys():
#         counter[key] += 1

In [21]:
# plt.hist(list(counter.keys()), bins=20)
# plt.show()

In [22]:
# import pandas as pd

# test_data = pd.read_csv('./data/test.csv')
# train_data = pd.read_csv('./data/train.csv')

# train_data

# train_patients_data = defaultdict(list)

# for row in train_data.iterrows():
#     train_patients_data[row[1]['Patient']].append((
#         int(row[1]['Weeks']), 
#         float(row[1]['FVC']),
#         float(row[1]['Percent']),
#         int(row[1]['Age']),
#         row[1]['Sex'],
#         row[1]['SmokingStatus']
#     ))

# for patient, values in train_patients_data.items():
#     weeks, fvcs, percents, ages, sex, smoking = zip(*values)
#     plt.plot(weeks, sex)

In [23]:
# def get_CT_patient(mode, patient_n):
#     base_path = os.path.join(IMAGE_PATH, mode)
#     patient = sorted(os.listdir(base_path))[patient_n]
#     patient_path = os.path.join(base_path, patient)
    
#     all_images = []  # , all_lungs, all_residuals, all_masks = [], [], [], []
#     meta_data = {
#         'InstanceNumber': [], 'SliceLocation': [], 'SliceThickness': [], 'PixelSpacing': [], 'PatientPosition': []
#     }
#     for idx, ct_name in enumerate(sorted(os.listdir(patient_path), key=lambda x: int(x.split('.')[0]))):
#         ct_path = os.path.join(patient_path, ct_name)
#         ct, ct_image = transform_to_hu(dcmread(ct_path))
#         ct_image = transformations[mode][patient](ct_image)
#         all_images.append(ct_image)
        
# #         lungs, residual, mask = segment_lungs(ct_image, patient_n, display=False)
        
# #         all_lungs.append(lungs)
# #         all_residuals.append(residual)        
# #         all_masks.append(mask)
        
#         for key in meta_data.keys():
#             meta_data[key].append(getkey(ct, key))
            
#     return all_images, meta_data  # all_lungs, all_residuals, all_masks, 

In [24]:
# all_images, meta_data = get_CT_patient('train', 0)

# torch.stack(tuple(map(torch.tensor, all_images))).shape

In [25]:
# len(sorted(os.listdir(base_path)))

In [26]:
# patient = sorted(os.listdir(base_path))[patient_n]

In [27]:
# patient

In [28]:
DEFAULT_SHAPE = (64, 256, 256)

In [29]:
class CTDataset(Dataset):
    """CT dataset."""

    def __init__(self, mode, wanted_shape=DEFAULT_SHAPE):  # , transform=None):  # csv_file
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
#         self.landmarks_frame = pd.read_csv(csv_file)
        self.base_path = os.path.join(IMAGE_PATH, mode)
        self.df = pd.read_csv(self.base_path + '.csv')
        self.patients = sorted(os.listdir(self.base_path))
        self.wanted_shape = wanted_shape
        self.mode = mode
#         self.transform = transform

    def __len__(self):
        return len(self.patients)

    def __getitem__(self, patient_n):
        if torch.is_tensor(patient_n):
            patient_n = patient_n.tolist()

        patient = self.patients[patient_n]
        patient_path = os.path.join(self.base_path, patient)
        
#         print(patient)
        patient_df = self.df[self.df.Patient == patient]
        min_week = patient_df.Weeks.min()
        base_FVC = patient_df.FVC[patient_df.Weeks == min_week].values
        base_df = patient_df[patient_df.Weeks == min_week]
        if len(base_FVC) != 1:
            print('More than 1 FVC measurment on min week')
        base_FVC = base_FVC[0]
        rest_df = patient_df[patient_df.Weeks != min_week]
#         print(patient, min_week, base_FVC)

        all_images = []  # , all_lungs, all_residuals, all_masks = [], [], [], []
        meta_data = {
            'InstanceNumber': [], 'SliceLocation': [], 'SliceThickness': [], 'PixelSpacing': [], 'PatientPosition': []
        }
        for idx, ct_name in enumerate(sorted(os.listdir(patient_path), key=lambda x: int(x.split('.')[0]))):
            ct_path = os.path.join(patient_path, ct_name)
            ct, ct_image = transform_to_hu(dcmread(ct_path))
            ct_image = transformations[self.mode][patient](ct_image)
            all_images.append(ct_image)

#             lungs, residual, mask = segment_lungs(ct_image, patient_n, display=False)

#             all_lungs.append(lungs)
#             all_residuals.append(residual)        
#             all_masks.append(mask)

            for key in meta_data.keys():
                meta_data[key].append(getkey(ct, key))

#         all_images, meta_data  # all_lungs, all_residuals, all_masks, 

        images = np.array(all_images)
        factor = [w / float(f) for w, f in zip(self.wanted_shape, images.shape)]
        resized = nd.interpolation.zoom(images, zoom=factor)
        image3D = torch.from_numpy(resized).unsqueeze(0).float()

#         if self.transform:
#             image3D = self.transform(image3D)
        
        return {
            'image': image3D,
#             'base_df': base_df,
#             'rest_df': rest_df,
#             'meta_data': meta_data
        }



#         img_name = os.path.join(self.root_dir,
#                                 self.landmarks_frame.iloc[idx, 0])
#         image = io.imread(img_name)
#         landmarks = self.landmarks_frame.iloc[idx, 1:]
#         landmarks = np.array([landmarks])
#         landmarks = landmarks.astype('float').reshape(-1, 2)
#         sample = {'image': image, 'landmarks': landmarks}

#         return sample


In [30]:
train_dataset = CTDataset('train')

In [31]:
# train_dataset.df

In [32]:
tmp = train_dataset[0]

In [33]:
tmp['image'].shape

torch.Size([1, 64, 256, 256])

In [34]:
# train_df[train_df.Patient == 'ID00009637202177434476278']

In [35]:
# test_df

In [36]:
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=0)

In [37]:
for batch in train_dataloader:
    break

In [38]:
batch['image'].shape

torch.Size([1, 1, 64, 256, 256])

In [39]:
# from efficientnet_pytorch import EfficientNet

# model = EfficientNet.from_pretrained('efficientnet-b0', in_channels=1)

In [40]:
# for i in range(1, 300):
#     tmp = torch.randn(2, 1, i, 32)
#     res = model.extract_features(tmp).shape
#     print(i, res[2])
# #     break

In [41]:
model = EfficientNet3D.from_name("efficientnet-b0", override_params={'num_classes': 2}, in_channels=1)

In [42]:
summary(model, input_size=(1,) + DEFAULT_SHAPE);

Layer (type:depth-idx)                             Param #
├─Conv3dStaticSamePadding: 1-1                     --
|    └─ZeroPad2d: 2-1                              --
├─BatchNorm3d: 1-2                                 64
├─ModuleList: 1-3                                  --
|    └─MBConvBlock3D: 2-2                          --
|    |    └─Conv3dStaticSamePadding: 3-1           864
|    |    └─BatchNorm3d: 3-2                       64
|    |    └─Conv3dStaticSamePadding: 3-3           264
|    |    └─Conv3dStaticSamePadding: 3-4           288
|    |    └─Conv3dStaticSamePadding: 3-5           512
|    |    └─BatchNorm3d: 3-6                       32
|    |    └─MemoryEfficientSwish: 3-7              --
|    └─MBConvBlock3D: 2-3                          --
|    |    └─Conv3dStaticSamePadding: 3-8           1,536
|    |    └─BatchNorm3d: 3-9                       192
|    |    └─Conv3dStaticSamePadding: 3-10          2,592
|    |    └─BatchNorm3d: 3-11                      192
|    |    └

In [43]:
# model = model.cuda()


# inputs = torch.randn((1, 3, 200, 200, 200)).cuda()
# labels = torch.tensor([0]).cuda()
# test forward
# num_classes = 2

In [44]:
model

EfficientNet3D(
  (_conv_stem): Conv3dStaticSamePadding(
    1, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), bias=False
    (static_padding): ZeroPad2d(padding=(0, 1, 0, 1, 0, 1), value=0.0)
  )
  (_bn0): BatchNorm3d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock3D(
      (_depthwise_conv): Conv3dStaticSamePadding(
        32, 32, kernel_size=(3, 3, 3), stride=[1, 1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d(padding=(1, 1, 1, 1, 1, 1), value=0.0)
      )
      (_bn1): BatchNorm3d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv3dStaticSamePadding(
        32, 8, kernel_size=(1, 1, 1), stride=(1, 1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv3dStaticSamePadding(
        8, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv3dStaticS

In [45]:
batch['image'].dtype, batch['image'].shape

(torch.float32, torch.Size([1, 1, 64, 256, 256]))

In [46]:
res = model.extract_features(batch['image'])

In [47]:
res.shape

torch.Size([1, 1280, 2, 8, 8])

In [167]:
device = 'cuda:0'

In [ ]:
model = model.to(device)

In [168]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=5e-4)

In [169]:
for i in range(10):
    x_tmp = torch.randn(1, 1, 300, 256, 256)
    res = model.extract_features(x_tmp.to(device))
    loss = res.sum()
    loss.backward()
    optimizer.step()

In [ ]:
def resample(image, target_shape, SliceThickness, PixelSpacing):
    factors = np.array(target_shape, dtype=np.float32) / np.array(image.shape, dtype=np.float32)
    image = scipy.ndimage.interpolation.zoom(image, factors, mode='nearest')

    # Determine current pixel spacing
    spacing = np.array([SliceThickness] + list(PixelSpacing), dtype=np.float32)
    new_spacing = spacing / factors

    return image, new_spacing

In [133]:
from torch import nn

In [ ]:
class Net(nn.Module):
    def __init__(self, efficient_net_number=0):
        super().__init__()
        self.model = EfficientNet3D.from_name(f"efficientnet-b{efficient_net_number}", in_channels=1)
        # override_params={'num_classes': 2}

    def forward(self, x):
        x = self.model(x)

In [70]:
model(batch['image'])

tensor([[0.0406, 0.0494]], grad_fn=<AddmmBackward>)

In [138]:
NUM_SLICES = 64
cur_slices = 900
if cur_slices > 64:
    tmp = np.linspace(0, cur_silces, NUM_SLICES)

In [140]:
tmp.shape

(64,)

In [22]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [23]:
inputs.shape

torch.Size([1, 3, 200, 200, 200])

In [16]:
model.train()
for epoch in range(2):
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    print('[%d] loss: %.3f' % (epoch + 1, loss.item()))

print('Finished Training')

[1] loss: 0.973
[2] loss: 0.848
Finished Training


In [246]:
mode = 'train'
wanted_shape = DEFAULT_SHAPE  # (20, 256, 256)

base_path = os.path.join(IMAGE_PATH, mode)
for patient_n in range(len(os.listdir(base_path))):
    all_images, meta_data = get_CT_patient(mode, patient_n)
#     X = torch.stack(tuple(map(torch.tensor, all_images))).shape
    images = np.array(all_images)
    factor = [w / float(f) for w, f in zip(wanted_shape, images.shape)]
    resized = nd.interpolation.zoom(images, zoom=factor)
    return torch.from_numpy(resized)

SyntaxError: 'return' outside function (<ipython-input-246-d875468caf7f>, line 11)

In [ ]:
# transforms.Resize((20, 256, 256))

In [ ]:
dsfactor = [w / float(f) for w, f in zip(whole.shape, flash.shape)]
downed = nd.interpolation.zoom(flash, zoom=dsfactor)


In [55]:
nd.interpolation.zoom()

<function scipy.ndimage.interpolation.zoom(input, zoom, output=None, order=3, mode='constant', cval=0.0, prefilter=True)>